In [1]:
import math 
from ROOT import TMVA, TFile, TString
from array import array
import ROOT


Welcome to JupyROOT 6.18/00


In [10]:
%jsroot on

In [2]:
f1 = ROOT.TFile.Open('/eos/user/g/gtolkach/data/tau_mu_plusandminus.root')
oldtree = f1.Get('NOMINAL')

In [21]:
#ULong64_t event
#N = background.GetEntries()
#EvaluateMVAB = array('f',N*[0.])
#event = array('i',[0])

#f11 = ROOT.TFile("EvaluateMVATest.root","recreate")
#EvMva = ROOT.TTree("EvMva","Simple Tree")
#EvMva.Branch("EventBranch", event, "EventBranch/I")
#EvMva.Branch("EvaluateMVAB", EvaluateMVAB, "EvaluateMVAB[EventBranch]/F")

In [3]:
from ROOT import TFile, TTree
from ROOT import gROOT, AddressOf

gROOT.ProcessLine(
"struct MyStruct {\
   Int_t     fEvent;\
   Double_t    fEvMva;\
};" );
 
from ROOT import MyStruct
mystruct = MyStruct()
 
f2 = TFile( 'treeEvMVA_for_data_mu.root', 'RECREATE' )
tree = TTree( 'EvMvatree', 'Just A Tree' )
tree.Branch( 'event', mystruct, 'fEvent/I' )
tree.Branch( 'EvMva', AddressOf( mystruct, 'fEvMva' ), 'EvMva/F' )

<ROOT.TBranch object ("EvMva") at 0x5e9af60>

In [3]:
TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()
reader = TMVA.Reader( "!Color:!Silent" )


In [4]:

#branches = {}
#for branch in signal.GetListOfBranches():
#    branchName = branch.GetName()
#    branches[branchName] = array('f', [-999])
#    signal.SetBranchAddress(branchName, branches[branchName])
#    background.SetBranchAddress(branchName, branches[branchName])
#    if branchName != 'EventBranch':
#        reader.AddVariable(branchName, branches[branchName])

In [5]:
Pt = array('f',[0])
metE = array('f',[0])
dPhi = array('f',[0])
PhiMiss = array('f',[0])
Mt = array('f',[0])
Eta = array('f',[0])
n = oldtree.GetEntries()

#for i in range(0,n):
#	oldtree.GetEntry(i)
#	PT = oldtree.GetLeaf("lep_0_pr.Pt()")

In [18]:
reader.AddVariable( "Pt", Pt)
oldtree.SetBranchAddress("", Pt )

reader.AddVariable( "metE ",metE)
oldtree.SetBranchAddress("met_reco_p4.E()", metE )

reader.AddVariable( "dPhi ",dPhi)
oldtree.SetBranchAddress("lepmet_dphi", dPhi )

reader.AddVariable( "PhiMiss ",PhiMiss)
oldtree.SetBranchAddress("met_reco_p4.Phi()", PhiMiss )

reader.AddVariable( "Mt", Mt)
oldtree.SetBranchAddress("lepmet_mt", Mt )

reader.AddVariable( "Eta",Eta)
oldtree.SetBranchAddress("lep_0_p4.Eta()", Eta )



Error in <TTree::SetBranchAddress>: unknown branch -> lep_0_p4.Pt()


In [25]:
reader.BookMVA("MLP",ROOT.TString("/eos/user/g/gtolkach/SWAN_projects/testpy/datasetV6newBDT/weights/TMVAClassification_MLP.weights.xml"))

<ROOT.TMVA::MethodMLP object ("MLP") at 0x546d200>

                         : Booking "MLP" of type "MLP" from /eos/user/g/gtolkach/SWAN_projects/testpy/datasetV6newBDT/weights/TMVAClassification_MLP.weights.xml.
                         : Reading weight file: /eos/user/g/gtolkach/SWAN_projects/testpy/datasetV6newBDT/weights/TMVAClassification_MLP.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
<HEADER> MLP                      : Building Network. 
                         : Initializing weights
                         : Booked classifier "MLP" of type: "MLP"


In [31]:
h = ROOT.TH1D("output_s","Blue=Signal, Red=Background;EvaluateMVA;Probability", 20,0, 1.1)

In [23]:
for i in range(0,n):
    oldtree.GetEntry(i)
    Pt[0] = oldtree.Pt
    MetE[0] = oldtree.metE
    dPhi[0] = oldtree.dPhi
    Mt[0] = oldtree.Mt
    Eta[0] = oldtree.Eta
    PhiMiss[0] = oldtree.PhiMiss
    l = reader.EvaluateMVA("MLP")
    h.Fill(l)
#    for write tree
#    mystruct.fEvent = i
#    mystruct.fEvMva = reader.EvaluateMVA("MLP")
#    tree.Fill()

AttributeError: 'TTree' object has no attribute 'PT'

In [ ]:
f2.Write()

In [ ]:
c1 = ROOT.TCanvas() 
c1.cd()
h.Draw("hist ")
c1.Draw()

1450

In [12]:
f1.Close()
f2.Close

In [1]:
#f2 = ROOT.TFile.Open('/eos/user/g/gtolkach/background_event_with_cuts_plus_1.root')
#f2 = ROOT.TFile.Open('/eos/user/g/gtolkach/background_event_with_cuts_plus_1.root','UPDATE')
#tree = f2.Get('NOMINAL')
#tree.AddFriend("EvMvatree","treefriend.root")
#f2.Write


In [2]:
#df = ROOT.ROOT.Experimental.TDataFrame("tree", "treefriend.root");
#df.Snapshot("EvMVA", "/eos/user/g/gtolkach/background_event_with_cuts_plus_1.root", {"EvMva"})

In [3]:
#new tree
f1 = ROOT.TFile.Open('/eos/user/g/gtolkach/background_event_with_cuts_plus_1.root')
f2 = ROOT.TFile.Open('/eos/user/g/gtolkach/SWAN_projects/testpy/treefriend.root')
oldtree1 = f1.Get('NOMINAL')
oldtree2 = f2.Get('EvMvatree')


In [6]:
f = ROOT.TFile( 'newtree.root', 'RECREATE')
oldtree1.CloneTree().Write()
oldtree2.CloneTree().Write()
f.Close()

In [13]:
#oldtree.AddFriend(newtree)
#oldtree.AddFriend('EvMvatree','/eos/user/g/gtolkach/SWAN_projects/testpy/treefriend.root');

In [14]:
#oldtree.Write()

0

Error in <TFile::WriteTObject>: Directory treefriend.root is not writable
